# Posterior of token-level parameters

This notebook computes the sufficient statistics for token-level parameters from (partially) matched name lists. Since all token parameters are independent given matches and global prior parameters, this can be done for each token individually.

The likelihood can be computed directly from the Beta-Binomial pmf,

$$
p(k \mid n, \alpha, \beta) = \frac{Z(\alpha + k, \beta + n - k)}{Z(\alpha, \beta)}
$$

where

$$
Z(\alpha, \beta) = \frac{\Gamma(\alpha)\Gamma(\beta)}{\Gamma(\alpha + \beta)}
$$

is the normalizer of the Beta-distribution.

We use a mean-strength parametrization

$$
\begin{align}
\alpha &= s \pi \\
\beta &= s (1 - \pi)
\end{align}
$$

$\pi \in [0, 1]$ is an the prior probability of an unseen entity to contain the token in its name, so that it uses entities as the denominator (not names). Consequently, the estimate changes when names are matched to entities. It is also weakly correlated with $s$. This tends to be relatively well determined by the data except for very rare tokens.

$s > 0$ measures how consistently a token is used: For $s \rightarrow 0$ we expect a token to occur with certainty in all names of an entity if it has occured in one name. For $s \rightarrow \infty$ the frequency is the population frequency regardless of whether the token was part of a name of an entity or not. Hence, low values of $s$ correspond to more relevant tokens, while tokens with large values are effectively ignored.

We impose a log-normal prior on $s$ and a flat prior on $\log\pi$, since $\pi$ is much better determined by the data, while for $s$ the likelihood is not even a normalizable distribution in some cases.


In principle, the likelihood contains one factor per entity, but in practice there are large multiplicities. For pairwise matches, the only possible values of the sufficient statistics are $n = 1, k = 0,1$ and $n=2, k=0,1,2$. We compute the likelihood for the distinct sufficient statistics only once while taking multiplicities into account.

In [ ]:
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

from ptfidf import utils as ut
from ptfidf.train.aggregation import get_group_statistics, compress_group_statistics
from ptfidf.train.likelihood import beta_binomial_log_likelihood

In [ ]:
def label_training_data(names, matched_symbols):
    """label training data.
    
    Names with symbol in matched_symbols will be assigned to 
    the same entity (per symbol).
    
    Parameters
    ----------
    names : DataFrame
        columns ['source', 'symbol', ...]
    matched_symbols : sequence or int
        if sequence, these symbols are matched. If int,
        draw matched_symbols symbols randomly.
    Returns
    -------
    DataFrame
        with added column 'entity', which is '{symbol}' or
        '{source}:{symbol}' for matched respectively unmatched
        symbols.
    """
    if isinstance(matched_symbols, int):
        matched_symbols = np.random.choice(np.unique(names['symbol']), replace=False, size=matched_symbols)
    res = pd.Series(index=names.index, name='entity')
    idx = names['symbol'].isin(matched_symbols)
    res.loc[idx] = names.loc[idx, 'symbol']
    res.loc[~idx] = names.loc[~idx, 'source'] + ':' + names.loc[~idx, 'symbol']
    return res

# Preprocessing

## Read name files and vectorize

In [ ]:
# read files and concatenate into single table
datadir = Path('../data').resolve()
filenames = ['wikipedia.csv', 'slickcharts.csv']
names = pd.concat([
    pd.read_csv(datadir.joinpath(fn)).assign(source=fn.split('.')[0]) for fn in filenames
], axis=0).sort_values(['symbol', 'source']).reset_index(drop=True)

# vectorize
vec = CountVectorizer(binary=True).fit(names['name'])
X = vec.transform(names['name'])
vocabulary_inv = {v: k for k, v in vec.vocabulary_.items()}

## Label training data

Since there are many tokens with the same statistics (e.g. tokens that occur only once or twice), also compute distinct token statistics for more convenient display of interesting examples.

In [ ]:
labels = label_training_data(names, np.unique(names['symbol']))
y = LabelEncoder().fit_transform(labels.values)
counts, nobs = get_group_statistics(X, y)
token_stats = compress_group_statistics(counts, nobs)

# get distinct token statistics and associated tokens
distinct = (
    token_stats
    .groupby('token')
    .apply(lambda df: tuple(df[['n', 'k', 'weight']].apply(tuple, axis=1).sort_values()))
    .reset_index()
    .groupby(0)['token']
    .agg(lambda x: tuple(x)))

# Posterior Visualization

Show joint distribution of token-level parameters for selected token

## Set parameters

In [ ]:
prior_mean = np.log(.2)
prior_std = 2.
max_token_display = 5


pivals = np.linspace(np.log(1e-5), np.log(.5), 200)
svals = np.linspace(np.log(1e-4), np.log(1e3), 200)
s, pi = np.meshgrid(svals, pivals)
alpha, beta = np.exp(s) * np.exp(pi), np.exp(s) * (1. - np.exp(pi))

## Posterior computation and visualization

The posterior is computed on a grid of values.

In [ ]:
i = np.random.choice(distinct.size)
selected_tokens = distinct.iloc[i]

output = ', '.join(vocabulary_inv[t] for t in selected_tokens[:max_token_display])
if len(selected_tokens) > max_token_display:
    output += ', ... ({} more)'.format(len(selected_tokens) - max_token_display)
print('tokens:\n' + output)

ms = token_stats[token_stats['token'] == selected_tokens[0]].sort_values(['n', 'k'])[['k', 'n', 'weight']].copy()

res = beta_binomial_log_likelihood(
    alpha[..., None], beta[..., None],
    ms['k'].values, ms['n'].values).dot(ms['weight'].values
)
res -= .5 * (s - prior_mean)**2 / prior_std**2
res = np.exp(res - res.max())
res /= res.sum()

idx_smp = np.random.choice(res.size, p=res.ravel(), size=1000)
s_smp, pi_smp = [np.exp(arr.ravel()[idx_smp]) for arr in [s, pi]]

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 4))

lg10 = np.log(10)
ax = axes[0]
ax.set_xlabel('$log_{10} s$')
ax.set_ylabel('$log_{10} \pi$')
ax.pcolormesh(s / lg10, pi / lg10, res, cmap=plt.cm.Blues)
ax.plot(np.log(s_smp[:20]) / lg10, np.log(pi_smp[:20]) / lg10, 'r.', alpha=.6)
ax.grid()

ax = axes[1]
ax.set_title('marginal distribution $\log_{10}s$')
ax.plot(svals / lg10, res.sum(axis=0))

ax = axes[2]
ax.set_title('marginal distribution $\log_{10}\pi$')
ax.plot(pivals / lg10, res.sum(axis=1))

ms.set_index(['n', 'k'])